# Playing with the tokenizer

There are 4,107 tokens in total:

- 4,096 for each possible 6-mer
- 5 for each possible input nucleotide (A, T, C, G and N). They are used as fallback when the sequence is not divisible by 6.
- 1 for padding (PAD)
- 1 for the mask (MASK)
- 1 for the entire sequence (CLS)
- 3 additional tokens, which I will mostly ignore

All summed up, 4,107 tokens, each represented by an integer.

In [1]:
from transformers import AutoTokenizer

Let's see how the tokenizer works by studying a few examples. Let's first load the models:

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "InstaDeepAI/nucleotide-transformer-v2-50m-multi-species", trust_remote_code=True
)

Let's tokenize the empty sequence. This should only contain the CLS token:

In [3]:
tokenizer([""])["input_ids"]

[[3]]

Neat. We see that NT uses integer 3 for CLS. Let's see now how the individual nucleotides are embedded by feeding the tokenizer five 1-nucleotide sequences. Since they are only 1 nucleotide long, it won't use any token representing 6-mer:

In [4]:
tokenizer(["A", "T", "C", "G", "N"])["input_ids"]

[[3, 4102], [3, 4103], [3, 4104], [3, 4105], [3, 4106]]

We see now that A = 4102, T = 4103, C = 4104, G = 4105 and N = 4106. Let's play a bit with the sequence length:

In [5]:
tokenizer(["A", "AA", "AAA", "AAAA", "AAAAA", "AAAAA", "AAAAAA"])["input_ids"]

[[3, 4102],
 [3, 4102, 4102],
 [3, 4102, 4102, 4102],
 [3, 4102, 4102, 4102, 4102],
 [3, 4102, 4102, 4102, 4102, 4102],
 [3, 4102, 4102, 4102, 4102, 4102],
 [3, 6]]

As soon as the length of the sequence is longer than 6, the tokenizer starts using tokens to represent an 6-mer. In this case, AAAAAA = 6. Let's see now how order matters:

In [6]:
tokenizer(["AAAAAAT", "TAAAAAA"])["input_ids"]

[[3, 6, 4103], [3, 1030, 4102]]

Interesting! This shows us that the tokenizer reads left-to-right in a greedy manner. It tries to split the sequence into 6-mers starting from the left. If there is any subsequence remaining, it will revert to individual nucleotides. Hence, it decomposes AAAAAAT as AAAAAA + T, and TAAAAAA as TAAAAA + A.

Let's look into the PAD token. As we have seen, the tokenizer depending on the size of inputs and how they are tokenized, the outputs can be different. The PAD token is a special token to ensure that all output tokens have the same size. We do this by specifying a padding strategy:

In [7]:
tokenizer(["AAAAAAAAAAAAAAAAAAAAAAAA", "AAAAAA", "A"], padding="longest")["input_ids"]

[[3, 6, 6, 6, 6], [3, 6, 1, 1, 1], [3, 4102, 1, 1, 1]]

The padding strategy was to find the length of the longest tokenized sequence, and add pad tokens to the shorter tokenized sequences until their length is the same. In this case, the longest sequence was `AAAAAAAAAAAAAAAAAAAAAAAA`, which is reduced to 4 tokens (4 × `AAAAAA`). The other two sequences were padded with the PAD token (1).

We can access all the tokens:

In [8]:
i = 0
for k, v in tokenizer.get_vocab().items():
    print(f"{k}:\t{v}")
    i += 1

    if i == 15:
        break

<unk>:	0
<pad>:	1
<mask>:	2
<cls>:	3
<eos>:	4
<bos>:	5
AAAAAA:	6
AAAAAT:	7
AAAAAC:	8
AAAAAG:	9
AAAATA:	10
AAAATT:	11
AAAATC:	12
AAAATG:	13
AAAACA:	14


In [9]:
len(tokenizer.get_vocab())

4107

The tokenizer contains tokens that were not considered in the NT article. Some of them are needed to make the model robust to artifacts, like the unknown token (e.g., if our sequence contained other [IUPAC nucleotides](https://en.wikipedia.org/wiki/Nucleic_acid_notation)). The vocabulary also contains tokens to specify the beginning and the end of the sequence, let's see if the tokenizer is able to use them.

In [10]:
tokenizer(["A"], add_special_tokens=True)["input_ids"]

[[3, 4102]]

It seems it doesn't.

Last, we should consider the maximum input size of the model. Transformer models cannot take sequences of any length. Specifically, the NT can consider at most sequences of 2,048 tokens (~12 kbp). We can make the tokenizer aware of this limit (`max_length=2048`) and instructing it to truncate longer sequences (`truncation=True`).

In [11]:
tokenizer(["A"], max_length=5000, truncation=True)["input_ids"]

[[3, 4102]]

Worked out example:

In [12]:
tokenizer(["ATGGTAGCTACATCATCT"], max_length=5000, truncation=True)["input_ids"]

[[3, 506, 3662, 1567]]